In [1]:
import tree_sitter_languages
from tree_sitter import Language, Parser


In [ ]:
%%bash

p


In [10]:
from python_minifier import minify

code = '''
import sys
import os
import json
sys.stdout.write(json.dumps(os.environ['PATH'].split(os.pathsep)))
'''

c = minify(code)

shell_template = '''{ex} -c "$(cat<<'EOF'
{code}
EOF
)"'''

print(shell_template.format(ex='python3', code=c))


python3 -c "$(cat<<'EOF'
import sys,os,json
sys.stdout.write(json.dumps(os.environ['PATH'].split(os.pathsep)))
EOF
)"

In [12]:
%%bash

python3 -c "$(cat<<'EOF'
import sys,os,json
sys.stdout.write(json.dumps(os.environ['PATH'].split(os.pathsep),indent=2))
EOF
)"


[
  "/home/tim/.local/share/micromamba/envs/py-default/bin",
  "/home/tim/.opam/4.09.0/bin",
  "/home/tim/.pyenv/shims",
  "/home/tim/.local/argc-completions/bin",
  "/home/tim/.local/share/micromamba/envs/py-default/bin",
  "/home/tim/.nvm/versions/node/v20.14.0/bin",
  "/home/tim/.rbenv/shims",
  "/home/tim/.rbenv/bin",
  "/home/tim/.pyenv/shims",
  "/home/tim/.pyenv/bin",
  "/home/tim/bin",
  "/home/tim/.local/bin",
  "/usr/local/bin",
  "/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-romkatv-SLASH-zsh-bench",
  "/home/tim/.opam/4.09.0/bin",
  "/home/tim/.pyenv/shims",
  "/home/tim/.local/argc-completions/bin",
  "/home/tim/.nvm/versions/node/v20.14.0/bin",


  "/home/tim/.rbenv/shims",
  "/home/tim/.rbenv/bin",
  "/home/tim/.pyenv/bin",
  "/home/tim/.local/share/pnpm",
  "/home/tim/.local/share/micromamba/condabin",
  "/home/tim/.script/bin",
  "/home/tim/.shell/bin",
  "/home/tim/.dotnet/tools",
  "/home/tim/.dotnet",
  "/home/tim/.yarn/bin",
  "/home/tim/.cargo/bin",
  "/opt/nvim/bin",
  "/usr/local/games",
  "/usr/local/go/bin",
  "/home/tim/.local/python-utils/bin",
  "/home/tim/.local/go/bin",
  "/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-reegnz-SLASH-jq-zsh-plugin/bin",
  "/snap/bin",
  "/usr/local/sbin",
  "/usr/bin",
  "/usr/sbin",
  "/usr/games",
  "/bin",
  "/sbin",
  "/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-reegnz-SLASH-jq-zsh-plugin/bin",
  "/home/tim/.opam/4.09.0/bin",
  "/home/tim/.pyenv/shims",
  "/home/tim/.local/argc-completions/bin",
  "/home/tim/.local/share/micromamba/envs/py-default/bin",
  "/home/tim/.nvm/versions/node/v20.14.0/bin",
  "/home/tim/.rbenv/shims",
  

In [13]:
"/usr/local/go/bin:/opt/nvim/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin".split(':')


['/usr/local/go/bin',
 '/opt/nvim/bin',
 '/usr/local/sbin',
 '/usr/local/bin',
 '/usr/sbin',
 '/usr/bin',
 '/sbin',
 '/bin',
 '/snap/bin']

In [1]:
import os
import sys
from typing import Iterable, Literal, Optional


class PathValue:
    def __init__(self, items: list[str] | str , pathsep: str = os.pathsep):
        self.__items: list[str] = items if isinstance(items, list) else items.split(pathsep)
        self.__pathsep = pathsep

    @property
    def lst(self) -> list[str]:
        return self.__items

    @property
    def str(self) -> str:
        return os.pathsep.join(self.__items)
    
    def __lst__(self):
        return self.__items
    
    def __str__(self):
        return self.str


class PathUtil:

    __sys_path: list[str] = [
        "/usr/local/sbin",
        "/usr/local/bin",
        "/usr/sbin",
        "/usr/bin",
        "/sbin",
        "/bin",
        "/snap/bin",
    ]

    def __init__(self, path: Optional[str | Iterable[str]] = None, pathsep: str = os.pathsep, **kwargs):

        self.__pathsep = pathsep 
        self.__change_log: list[tuple] = [] if not kwargs.get("change_log") else kwargs["log"]
    
        if not path:
            self.__path = PathValue(os.environ['PATH'], pathsep)
        elif isinstance(path, (str, list)):
            self.__path = PathValue(path, pathsep)
        else:
            raise ValueError("Invalid path value")
        
        self.__change_log.append(('init',self.__path))

    def __register_change(self, change_type:str, path: str|list[str]) -> None:
        p = PathValue(path, self.__pathsep)
        self.__change_log.insert(0, (change_type, p))

    @staticmethod
    def _split_path(value: str, pathsep: str = os.pathsep) -> list[str]:
        return value.split(pathsep)
    
    @property
    def path(self) -> PathValue:
        return self.__path
    
    @property
    def string(self) -> str:
        return self.__path.str
    
    @property
    def lst(self) -> list[str]:
        return self.__path.lst
    
    def __iter__(self):
        return iter(self.__path.lst)
    
    def __len__(self):
        return len(self.__path.lst)
    
    def __str__(self):
        return self.__path.str
    
    def __lst__(self):
        return self.__path.lst


    def find_duplicates(self) -> list[str]:
        fresh: set[str] = set()
        duplicates = []
        for item in self:
            if item in fresh:
                if item not in duplicates:
                    duplicates.append(item)
            else:
                fresh.add(item)
        return duplicates

    def __remove_duplicates(self) -> list[str]:
        fresh: list[str] = []
        for item in self:
            if item not in fresh:
                fresh.append(item)
        return fresh
    
    def remove_duplicates(self) -> None:
        fresh = self.__remove_duplicates()
        self.__register_change('remove_duplicates', fresh)
    
    def __find_invalid(self) -> list[str]:
        invalid: set[str] = set()
        for item in self:
            if not os.path.isdir(item):
                invalid.add(item)
        return list(invalid)
    
    def remove_invalid(self) -> None:
        invalid_items = self.__find_invalid()
        new_path = [item for item in self if item not in invalid_items]
        self.__register_change('remove_invalid', new_path)

    def ensure_sys_path_order(self) -> None:
        new_path = [p for p in self if p not in self.__sys_path]
        new_path.extend(self.__sys_path)
        self.__register_change('ensure_sys_path_order', new_path)


    def revert_change(self, steps: int = 1) -> None:
        if steps > len(self.__change_log):
            raise ValueError("Invalid steps")
        for _ in range(steps):
            self.__change_log.pop()
        self.__path = self.__change_log[-1][1]

    def remove_path(self, path: str) -> None:
        new_path = [p for p in self if p != path]
        self.__register_change(f'remove_path({path})', new_path)

    def add_path(self, path: str, meth:Literal['prepend', 'append']='prepend') -> None:
        new_path = self.lst
        if meth == 'prepend':
            new_path.append(path)
        elif meth == 'append':
            new_path.append(path)
        else:
            raise ValueError(f"Invalid method: {meth}. Use 'prepend' or 'append'")
        self.__register_change(f'add_path({path})', new_path)


p = PathUtil()

p.remove_duplicates()
p.remove_invalid()
p.ensure_sys_path_order()


/home/tim/.local/share/micromamba/envs/py-default/bin:/home/tim/.opam/4.09.0/bin:/home/tim/.pyenv/shims:/home/tim/.local/argc-completions/bin:/home/tim/.local/share/micromamba/envs/py-default/bin:/home/tim/.local/share/pnpm:/home/tim/.nvm/versions/node/v20.14.0/bin:/home/tim/.rbenv/shims:/home/tim/.rbenv/bin:/home/tim/.pyenv/shims:/home/tim/.pyenv/bin:/home/tim/.local/share/micromamba/condabin:/home/tim/bin:/home/tim/.local/bin:/usr/local/bin:/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-romkatv-SLASH-zsh-bench:/home/tim/.script/bin:/home/tim/.shell/bin:/home/tim/.dotnet/tools:/home/tim/.dotnet:/home/tim/.yarn/bin:/home/tim/.cargo/bin:/opt/nvim/bin:/usr/local/games:/usr/local/go/bin:/home/tim/.local/python-utils/bin:/home/tim/.pyenv/bin:/home/tim/.local/go/bin:/snap/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/usr/games:/bin:/sbin:/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-reegnz-SLASH-jq-zsh-plugin/bin:/home/tim/.opam/4.09.0/bin:/home/tim/.p

In [3]:
import os

from rich import print

print(os.environ['PATH'].split(os.pathsep))


[
    '/home/tim/.local/share/micromamba/envs/py-default/bin',
    '/home/tim/.opam/4.09.0/bin',
    '/home/tim/.pyenv/shims',
    '/home/tim/.local/argc-completions/bin',
    '/home/tim/.local/share/micromamba/envs/py-default/bin',
    '/home/tim/.nvm/versions/node/v20.14.0/bin',
    '/home/tim/.rbenv/shims',
    '/home/tim/.rbenv/bin',
    '/home/tim/.pyenv/shims',
    '/home/tim/.pyenv/bin',
    '/home/tim/bin',
    '/home/tim/.local/bin',
    '/usr/local/bin',
    '/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-romkatv-SLASH-zsh-bench',
    '/home/tim/.opam/4.09.0/bin',
    '/home/tim/.pyenv/shims',
    '/home/tim/.local/argc-completions/bin',
    '/home/tim/.nvm/versions/node/v20.14.0/bin',
    '/home/tim/.rbenv/shims',
    '/home/tim/.rbenv/bin',
    '/home/tim/.pyenv/bin',
    '/home/tim/.local/share/pnpm',
    '/home/tim/.local/share/micromamba/condabin',
    '/home/tim/.script/bin',
    '/home/tim/.shell/bin',
    '/home/tim/.dotnet/tools',
    '/home/tim/.dotnet',
    '/home/tim/.yarn/bin',
    '/home/tim/.cargo/bin',
    '/opt/nvim/bin',
    '/usr/local/games',
    '/usr/local/go/bin',
    '/home/tim/.local/python-utils/bin',
    '/home/tim/.local/go/bin',
    '/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-reegnz-SLASH-jq-zsh-plugin/bin',
    '/snap/bin',
    '/usr/local/sbin',
    '/usr/bin',
    '/usr/sbin',
    '/usr/games',
    '/bin',
    '/sbin',
    '/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-reegnz-SLASH-jq-zsh-plugin/bin',
    '/home/tim/.opam/4.09.0/bin',
    '/home/tim/.pyenv/shims',
    '/home/tim/.local/argc-completions/bin',
    '/home/tim/.local/share/micromamba/envs/py-default/bin',
    '/home/tim/.nvm/versions/node/v20.14.0/bin',
    '/home/tim/.rbenv/shims',
    '/home/tim/.rbenv/bin',
    '/home/tim/.pyenv/shims',
    '/home/tim/.pyenv/bin',
    '/home/tim/bin',
    '/home/tim/.local/bin',
    '/usr/local/bin',
    '/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-romkatv-SLASH-zsh-bench',
    '/home/tim/.opam/4.09.0/bin',
    '/home/tim/.pyenv/shims',
    '/home/tim/.local/argc-completions/bin',
    '/home/tim/.nvm/versions/node/v20.14.0/bin',
    '/home/tim/.rbenv/shims',
    '/home/tim/.rbenv/bin',
    '/home/tim/.pyenv/bin',
    '/home/tim/.local/share/pnpm',
    '/home/tim/.local/share/micromamba/condabin',
    '/home/tim/.script/bin',
    '/home/tim/.shell/bin',
    '/home/tim/.dotnet/tools',
    '/home/tim/.dotnet',
    '/home/tim/.yarn/bin',
    '/home/tim/.cargo/bin',
    '/opt/nvim/bin',
    '/usr/local/games',
    '/usr/local/go/bin',
    '/home/tim/.local/python-utils/bin',
    '/home/tim/.local/go/bin',
    '/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-reegnz-SLASH-jq-zsh-plugin/bin',
    '/snap/bin',
    '/usr/local/sbin',
    '/usr/bin',
    '/usr/sbin',
    '/usr/games',
    '/bin',
    '/sbin',
    '/home/tim/.cache/antidote/https-COLON--SLASH--SLASH-github.com-SLASH-reegnz-SLASH-jq-zsh-plugin/bin'
]